In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from flip import fisher, utils, data_vector, __flip_dir_path__
from flip.covariance import covariance


flip_base = Path(__flip_dir_path__)
data_path = flip_base / 'data'

[ 000000.00 ]: 06-09 10:29  root            INFO     No pypower module detected, gridding with this method is unavailable
[ 000000.11 ]: 06-09 10:29  jax._src.xla_bridge INFO     Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
[ 000000.12 ]: 06-09 10:29  jax._src.xla_bridge INFO     Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


In [2]:
grid = pd.read_parquet(data_path / "density_data.parquet")
grid_window = pd.read_parquet(data_path / "grid_window_m.parquet")

ktt, ptt = np.loadtxt(data_path / "power_spectrum_tt.txt")
kmt, pmt = np.loadtxt(data_path / "power_spectrum_mt.txt")
kmm, pmm = np.loadtxt(data_path / "power_spectrum_mm.txt")

sigmag_fiducial = 3.0

power_spectrum_dict_bias = {"gg": [[kmm, pmm * np.array(grid_window["window_mm"])**2]]}
power_spectrum_dict = {
    "gg": [
        [kmm, pmm * np.array(grid_window["window_mm"])**2],
        [kmt, pmt * np.array(grid_window["window_mt"])],
        [ktt, ptt]
    ]
                      }

In [3]:
DataFisher = data_vector.FisherDens(grid.rename(columns={'density_err': 'density_error', 'rcom': 'rcom_zobs'}).to_dict(orient='list'))
print("Data free parameters: ", DataFisher.free_par)

Data free parameters:  []


In [4]:
size_batch=10_000
number_worker=8
covariance_fit = DataFisher.compute_covariance("adamsblake20", power_spectrum_dict, size_batch=size_batch, 
                                              number_worker=number_worker, additional_parameters_values=(sigmag_fiducial,))

print("Covariance free parameters: ", covariance_fit.free_par)

/hpc/group/cosmology/blc56/miniconda3/envs/flip/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
[ 000015.00 ]: 06-09 10:30  root            INFO     Covariance matrix generated from flip with adamsblake20 model in 1.58e+01 seconds
[ 000015.00 ]: 06-09 10:30  root            INFO     Use self.init_covariance_sum before using self.compute_covariance_sum.


Covariance free parameters:  ['bs8', 'beta_f']


In [5]:
fisher_properties = {
    "inversion_method": "inverse",
}


In [6]:
parameter_dict = {
    "bs8": 0.8,
    "beta_f": 0.3,
}

Fisher = fisher.FisherMatrix.init_from_covariance(
    covariance_fit,
    DataFisher,
    parameter_dict,
    fisher_properties=fisher_properties,
)

parameter_name_list, fisher_matrix = Fisher.compute_fisher_matrix()

ERR = np.sqrt(np.diag(np.linalg.pinv(fisher_matrix)))

In [8]:
for p, e in zip(parameter_name_list, ERR):
    print(f"Fisher error on {p} = {e:.4f}")

Fisher error on fs8 = 0.0000
Fisher error on bs8 = 0.0322
Fisher error on beta_f = 0.0761
